### Visualizaciones de las predicciones del precio y consumo de la luz en un hogar

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.animation import FuncAnimation
import numpy as np
import matplotlib
from IPython.display import HTML
import matplotlib.animation as animation
from datetime import datetime
import os
import sys
from matplotlib.animation import FFMpegWriter

# Para notebooks: usar backend que funcione bien con animaciones
%matplotlib inline
from IPython.display import display
plt.rcParams['animation.html'] = 'jshtml'

# Estilo visual mejorado
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

# Cargar datos
df_precio = pd.read_csv("prediccion_precio.csv", parse_dates=["timestamp"])
df_consumo = pd.read_csv("predicciones_consumo.csv", parse_dates=["fecha"])

# Renombrar columnas para consistencia
df_precio.rename(columns={"€/kwh": "precio_real", "prediccion_€/kwh": "precio_pred"}, inplace=True)
df_consumo.rename(columns={"real_consumo": "consumo_real", "pred_consumo": "consumo_pred"}, inplace=True)

# ----------- MÉTODO 1: Usando FuncAnimation pero con configuración especial para notebooks -----------

def create_precio_animation():
    fig1, ax1 = plt.subplots()

    # Mostrar siempre todos los datos reales desde el principio
    line1_real, = ax1.plot(df_precio["timestamp"], df_precio["precio_real"], 
                          label="Precio real", linewidth=2, color='blue')

    # La línea de predicción comienza vacía y se irá llenando
    line1_pred, = ax1.plot([], [], label="Predicción", linestyle="--", 
                          linewidth=2, color='red')

    ax1.set_title("Precio de la luz (€/kWh)", fontsize=14, fontweight='bold')
    ax1.set_xlabel("Tiempo", fontsize=12)
    ax1.set_ylabel("€/kWh", fontsize=12)

    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m %H:%M'))
    ax1.xaxis.set_major_locator(mdates.AutoDateLocator())
    fig1.autofmt_xdate()

    # Limitar ejes
    time_min = df_precio["timestamp"].min()
    time_max = df_precio["timestamp"].max()
    y_min = df_precio["precio_real"].min()
    y_max = df_precio["precio_real"].max()
    if not pd.isna(df_precio["precio_pred"]).all():
        y_min = min(y_min, df_precio["precio_pred"].min())
        y_max = max(y_max, df_precio["precio_pred"].max())

    y_margin = (y_max - y_min) * 0.1
    ax1.set_xlim(time_min, time_max)
    ax1.set_ylim(y_min - y_margin, y_max + y_margin)

    ax1.legend(loc='best')
    ax1.grid(True)

    def init_precio():
        line1_pred.set_data([], [])
        return line1_pred,

    def update_precio(frame):
        x = df_precio["timestamp"][:frame+1]
        y_pred = df_precio["precio_pred"][:frame+1]
        
        line1_pred.set_data(x, y_pred)
        return line1_pred,
    
    ani = FuncAnimation(fig1, update_precio, frames=len(df_precio), init_func=init_precio,
                        interval=200, blit=True, repeat=False)
    
    plt.close() # Evita mostrar la figura dos veces en el notebook
    return ani, fig1

def create_consumo_animation():
    fig2, ax2 = plt.subplots()

    line2_real, = ax2.plot(df_consumo["fecha"], df_consumo["consumo_real"], 
                          label="Consumo real", linewidth=2, color='green')
    line2_pred, = ax2.plot([], [], label="Predicción", linestyle="--", 
                          linewidth=2, color='purple')

    ax2.set_title("Consumo energético (kWh)", fontsize=14, fontweight='bold')
    ax2.set_xlabel("Tiempo", fontsize=12)
    ax2.set_ylabel("kWh", fontsize=12)

    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m %H:%M'))
    ax2.xaxis.set_major_locator(mdates.AutoDateLocator())
    fig2.autofmt_xdate()

    # Limitar ejes
    time_min = df_consumo["fecha"].min()
    time_max = df_consumo["fecha"].max()
    y_min = df_consumo["consumo_real"].min()
    y_max = df_consumo["consumo_real"].max()
    if not pd.isna(df_consumo["consumo_pred"]).all():
        y_min = min(y_min, df_consumo["consumo_pred"].min())
        y_max = max(y_max, df_consumo["consumo_pred"].max())

    y_margin = (y_max - y_min) * 0.1
    ax2.set_xlim(time_min, time_max)
    ax2.set_ylim(y_min - y_margin, y_max + y_margin)

    ax2.legend(loc='best')
    ax2.grid(True)

    def init_consumo():
        line2_pred.set_data([], [])
        return line2_pred,

    def update_consumo(frame):
        x = df_consumo["fecha"][:frame+1]
        y_pred = df_consumo["consumo_pred"][:frame+1]
        
        line2_pred.set_data(x, y_pred)
        return line2_pred,
    
    ani = FuncAnimation(fig2, update_consumo, frames=len(df_consumo), init_func=init_consumo,
                        interval=200, blit=True, repeat=False)
    
    plt.close() # Evita mostrar la figura dos veces en el notebook
    return ani, fig2

# Crear y mostrar animaciones para notebook

ani1, fig1 = create_precio_animation()
ani2, fig2 = create_consumo_animation()

#writer = FFMpegWriter(fps=5, metadata=dict(artist='JoeyBG'), bitrate=1800)
#ani1.save("precio_animacion_completa.mp4", writer=writer)
#ani2.save("consumo_animacion_completa.mp4", writer=writer)
ani1.save("precio_animacion.gif", writer="pillow", fps=5)
ani2.save("consumo_animacion.gif", writer="pillow", fps=5)


# Mostrar las animaciones en el notebook
print("Animación de precios:")
display(HTML(ani1.to_jshtml()))
print("\nAnimación de consumo:")
display(HTML(ani2.to_jshtml()))

    
    


In [ ]:
!pip install ffmpeg-python
!pip install ffmpeg

In [ ]:
!ffmpeg -version


### Predicciones para las primeras 48 horas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.animation import FuncAnimation
import numpy as np
import matplotlib
from IPython.display import HTML
import matplotlib.animation as animation
from datetime import datetime, timedelta
import os
import sys
from matplotlib.animation import FFMpegWriter

# Para notebooks: usar backend que funcione bien con animaciones
%matplotlib inline
from IPython.display import display
plt.rcParams['animation.html'] = 'jshtml'

# Estilo visual mejorado
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

# Cargar datos
df_precio = pd.read_csv("prediccion_precio.csv", parse_dates=["timestamp"])
df_consumo = pd.read_csv("predicciones_consumo.csv", parse_dates=["fecha"])

# Renombrar columnas para consistencia
df_precio.rename(columns={"€/kwh": "precio_real", "prediccion_€/kwh": "precio_pred"}, inplace=True)
df_consumo.rename(columns={"real_consumo": "consumo_real", "pred_consumo": "consumo_pred"}, inplace=True)

# ----------- FILTRAR DATOS SEGÚN LOS REQUISITOS -----------

# Para precios - Filtrar solo 2 días
precio_min_date = df_precio["timestamp"].min()
precio_max_date = precio_min_date + timedelta(days=5)  # Solo 2 días
df_precio = df_precio[df_precio["timestamp"] < precio_max_date]

# Separar los datos del primer día y segundo día para precio
first_day_end = precio_min_date + timedelta(days=4)
df_precio_dia1 = df_precio[df_precio["timestamp"] < first_day_end]
df_precio_dia2 = df_precio[df_precio["timestamp"] >= first_day_end]

# Para consumo - Filtrar solo 1 mes
consumo_min_date = df_consumo["fecha"].min()
consumo_max_date = consumo_min_date + timedelta(days=7)  # Aproximadamente 1 mes
df_consumo = df_consumo[df_consumo["fecha"] < consumo_max_date]

# ----------- MÉTODO 1: Usando FuncAnimation pero con configuración especial para notebooks -----------

def create_precio_animation():
    fig1, ax1 = plt.subplots()

    # Para el primer día: Mostrar siempre ambos, real y predicho
    line1_real_dia1, = ax1.plot(df_precio_dia1["timestamp"], df_precio_dia1["precio_real"], 
                               label="Precio real", linewidth=2, color='blue')
    line1_pred_dia1, = ax1.plot(df_precio_dia1["timestamp"], df_precio_dia1["precio_pred"], 
                               label="Predicción", linestyle="--", linewidth=2, color='red')
    
    # Para el segundo día: Solo mostrar la predicción
    line1_pred_dia2, = ax1.plot([], [], label="Predicción (día siguiente)", 
                              linestyle="--", linewidth=2, color='darkorange')

    ax1.set_title("Precio de la luz (€/kWh)", fontsize=14, fontweight='bold')
    ax1.set_xlabel("Tiempo", fontsize=12)
    ax1.set_ylabel("€/kWh", fontsize=12)

    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m %H:%M'))
    ax1.xaxis.set_major_locator(mdates.AutoDateLocator())
    fig1.autofmt_xdate()

    # Limitar ejes
    time_min = df_precio["timestamp"].min()
    time_max = df_precio["timestamp"].max()
    y_min = df_precio["precio_real"].min()
    y_max = df_precio["precio_real"].max()
    if not pd.isna(df_precio["precio_pred"]).all():
        y_min = min(y_min, df_precio["precio_pred"].min())
        y_max = max(y_max, df_precio["precio_pred"].max())

    y_margin = (y_max - y_min) * 0.1
    ax1.set_xlim(time_min, time_max)
    ax1.set_ylim(y_min - y_margin, y_max + y_margin)

    ax1.legend(loc='best')
    ax1.grid(True)
    
    # Resaltar la división entre el día 1 y el día 2
    ax1.axvline(x=first_day_end, color='gray', linestyle='-', alpha=0.5)
    ax1.text(first_day_end + timedelta(hours=1), 
             y_min + 0.5*(y_max-y_min), 
             'Día Siguiente: Solo predicción', 
             rotation=90, 
             verticalalignment='center')

    def init_precio():
        line1_pred_dia2.set_data([], [])
        return line1_pred_dia2,

    def update_precio(frame):
        if frame < len(df_precio_dia2):
            x = df_precio_dia2["timestamp"][:frame+1]
            y_pred = df_precio_dia2["precio_pred"][:frame+1]
            line1_pred_dia2.set_data(x, y_pred)
        return line1_pred_dia2,
    
    ani = FuncAnimation(fig1, update_precio, frames=len(df_precio_dia2), init_func=init_precio,
                        interval=500, blit=True, repeat=False)
    
    plt.close() # Evita mostrar la figura dos veces en el notebook
    return ani, fig1

def create_consumo_animation():
    fig2, ax2 = plt.subplots()

    # Para consumo: La predicción está siempre visible desde el principio
    line2_pred, = ax2.plot(df_consumo["fecha"], df_consumo["consumo_pred"], 
                         label="Predicción", linestyle="--", linewidth=2, color='red')
    
    # La línea de datos reales se va llenando gradualmente (invertimos la lógica)
    line2_real, = ax2.plot([], [], label="Consumo real", linewidth=2, color='blue')

    ax2.set_title("Consumo energético (kWh)", fontsize=14, fontweight='bold')
    ax2.set_xlabel("Tiempo", fontsize=12)
    ax2.set_ylabel("kWh", fontsize=12)

    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m %H:%M'))
    ax2.xaxis.set_major_locator(mdates.AutoDateLocator())
    fig2.autofmt_xdate()

    # Limitar ejes
    time_min = df_consumo["fecha"].min()
    time_max = df_consumo["fecha"].max()
    y_min = df_consumo["consumo_real"].min()
    y_max = df_consumo["consumo_real"].max()
    if not pd.isna(df_consumo["consumo_pred"]).all():
        y_min = min(y_min, df_consumo["consumo_pred"].min())
        y_max = max(y_max, df_consumo["consumo_pred"].max())

    y_margin = (y_max - y_min) * 0.1
    ax2.set_xlim(time_min, time_max)
    ax2.set_ylim(y_min - y_margin, y_max + y_margin)

    ax2.legend(loc='best')
    ax2.grid(True)

    # Añadir texto que indique que estamos mostrando un mes de datos
    ax2.text(time_min + (time_max - time_min) * 0.5, 
             y_max + y_margin * 0.5, 
             'Datos de un mes', 
             horizontalalignment='center',
             fontsize=10)

    def init_consumo():
        line2_real.set_data([], [])
        return line2_real,

    def update_consumo(frame):
        x = df_consumo["fecha"][:frame+1]
        y_real = df_consumo["consumo_real"][:frame+1]
        
        line2_real.set_data(x, y_real)
        return line2_real,
    
    ani = FuncAnimation(fig2, update_consumo, frames=len(df_consumo), init_func=init_consumo,
                        interval=100, blit=True, repeat=False)
    
    plt.close() # Evita mostrar la figura dos veces en el notebook
    return ani, fig2

# Crear y mostrar animaciones para notebook
ani1, fig1 = create_precio_animation()
ani2, fig2 = create_consumo_animation()

# Guardar animaciones como GIF (más compatible con notebooks)
#ani1.save("precio_animacion.gif", writer="pillow", fps=5)
ani1.save("precio_animacion2.gif", writer="pillow", fps=10)
ani2.save("consumo_animacion_7dias.gif", writer="pillow", fps=10)

# Intenta guardar como MP4 si FFMpeg está disponible
#try:
#    writer = FFMpegWriter(fps=5, metadata=dict(artist='Usuario'), bitrate=1800)
#    ani1.save("precio_animacion.mp4", writer=writer)
#    ani2.save("consumo_animacion.mp4", writer=writer)
#    print("Animaciones guardadas como MP4")
#except Exception as e:
#    print(f"No se pudieron guardar como MP4: {e}")
#    print("Solo se guardaron como GIF")
#
# Mostrar las animaciones en el notebook
print("Animación de precios (2 días, primer día completo, segundo día solo predicción):")
display(HTML(ani1.to_jshtml()))
print("\nAnimación de consumo (1 mes, predicción constante, datos reales se añaden gradualmente):")
display(HTML(ani2.to_jshtml()))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.animation import FuncAnimation
import numpy as np
import matplotlib
from IPython.display import HTML
import matplotlib.animation as animation
from datetime import datetime
import os
import sys
from matplotlib.animation import FFMpegWriter

# Para notebooks: usar backend que funcione bien con animaciones
%matplotlib inline
from IPython.display import display
plt.rcParams['animation.html'] = 'jshtml'

# Estilo visual mejorado
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

# Cargar datos
df_precio = pd.read_csv("prediccion_precio.csv", parse_dates=["timestamp"])
df_consumo = pd.read_csv("predicciones_consumo.csv", parse_dates=["fecha"])

# Renombrar columnas para consistencia
df_precio.rename(columns={"€/kwh": "precio_real", "prediccion_€/kwh": "precio_pred"}, inplace=True)
df_consumo.rename(columns={"real_consumo": "consumo_real", "pred_consumo": "consumo_pred"}, inplace=True)

# ----------- MÉTODO 1: Usando FuncAnimation pero con configuración especial para notebooks -----------

def create_precio_animation():
    fig1, ax1 = plt.subplots()

    # Mostrar siempre todos los datos reales desde el principio
    line1_real, = ax1.plot(df_precio["timestamp"], df_precio["precio_real"], 
                          label="Precio real", linewidth=2, color='blue')

    # La línea de predicción comienza vacía y se irá llenando
    line1_pred, = ax1.plot([], [], label="Predicción", linestyle="--", 
                          linewidth=2, color='red')

    ax1.set_title("Precio de la luz (€/kWh)", fontsize=14, fontweight='bold')
    ax1.set_xlabel("Tiempo", fontsize=12)
    ax1.set_ylabel("€/kWh", fontsize=12)

    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m %H:%M'))
    ax1.xaxis.set_major_locator(mdates.AutoDateLocator())
    fig1.autofmt_xdate()

    # Limitar ejes
    time_min = df_precio["timestamp"].min()
    time_max = df_precio["timestamp"].max()
    y_min = df_precio["precio_real"].min()
    y_max = df_precio["precio_real"].max()
    if not pd.isna(df_precio["precio_pred"]).all():
        y_min = min(y_min, df_precio["precio_pred"].min())
        y_max = max(y_max, df_precio["precio_pred"].max())

    y_margin = (y_max - y_min) * 0.1
    ax1.set_xlim(time_min, time_max)
    ax1.set_ylim(y_min - y_margin, y_max + y_margin)

    ax1.legend(loc='best')
    ax1.grid(True)

    def init_precio():
        line1_pred.set_data([], [])
        return line1_pred,

    def update_precio(frame):
        x = df_precio["timestamp"][:frame+1]
        y_pred = df_precio["precio_pred"][:frame+1]
        
        line1_pred.set_data(x, y_pred)
        return line1_pred,
    
    ani = FuncAnimation(fig1, update_precio, frames=len(df_precio), init_func=init_precio,
                        interval=200, blit=True, repeat=False)
    
    plt.close() # Evita mostrar la figura dos veces en el notebook
    return ani, fig1

def create_consumo_animation():
    fig2, ax2 = plt.subplots()

    line2_real, = ax2.plot(df_consumo["fecha"], df_consumo["consumo_real"], 
                          label="Consumo real", linewidth=2, color='green')
    line2_pred, = ax2.plot([], [], label="Predicción", linestyle="--", 
                          linewidth=2, color='purple')

    ax2.set_title("Consumo energético (kWh)", fontsize=14, fontweight='bold')
    ax2.set_xlabel("Tiempo", fontsize=12)
    ax2.set_ylabel("kWh", fontsize=12)

    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m %H:%M'))
    ax2.xaxis.set_major_locator(mdates.AutoDateLocator())
    fig2.autofmt_xdate()

    # Limitar ejes
    time_min = df_consumo["fecha"].min()
    time_max = df_consumo["fecha"].max()
    y_min = df_consumo["consumo_real"].min()
    y_max = df_consumo["consumo_real"].max()
    if not pd.isna(df_consumo["consumo_pred"]).all():
        y_min = min(y_min, df_consumo["consumo_pred"].min())
        y_max = max(y_max, df_consumo["consumo_pred"].max())

    y_margin = (y_max - y_min) * 0.1
    ax2.set_xlim(time_min, time_max)
    ax2.set_ylim(y_min - y_margin, y_max + y_margin)

    ax2.legend(loc='best')
    ax2.grid(True)

    def init_consumo():
        line2_pred.set_data([], [])
        return line2_pred,

    def update_consumo(frame):
        x = df_consumo["fecha"][:frame+1]
        y_pred = df_consumo["consumo_pred"][:frame+1]
        
        line2_pred.set_data(x, y_pred)
        return line2_pred,
    
    ani = FuncAnimation(fig2, update_consumo, frames=len(df_consumo), init_func=init_consumo,
                        interval=200, blit=True, repeat=False)
    
    plt.close() # Evita mostrar la figura dos veces en el notebook
    return ani, fig2

# Crear y mostrar animaciones para notebook

ani1, fig1 = create_precio_animation()
ani2, fig2 = create_consumo_animation()

#writer = FFMpegWriter(fps=5, metadata=dict(artist='JoeyBG'), bitrate=1800)
#ani1.save("precio_animacion_completa.mp4", writer=writer)
#ani2.save("consumo_animacion_completa.mp4", writer=writer)
ani1.save("precio_animacion.gif", writer="pillow", fps=5)
ani2.save("consumo_animacion.gif", writer="pillow", fps=5)


# Mostrar las animaciones en el notebook
print("Animación de precios:")
display(HTML(ani1.to_jshtml()))
print("\nAnimación de consumo:")
display(HTML(ani2.to_jshtml()))

    
    
